In [1]:
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from tqdm import tqdm
import pickle

from sklearn.linear_model import Ridge


In [2]:
directory = "/kaggle/input/adc24-intro-training/"

exec(open(directory + 'f_read_and_preprocess.py', 'r').read())
exec(open(directory + 'a_read_and_preprocess.py', 'r').read())
exec(open(directory + 'feature_engineering.py', 'r').read())
exec(open(directory + 'postprocessing.py', 'r').read())


The training data cover planets of only two stars (stars 0 and 1), but the test data include planets of other stars.

This leads to the following recipe:
- For known stars (stars 0 and 1), we expect the test rmse to be equal to our cross-validation rmse, i.e. we predict the out-of-fold rmse of our model (0.000293 as shown in the training notebook).
- For unknown stars, the prediction error can only be higher. We thus predict a higher value (0.001 in this notebook).

In [3]:
# Load the data
wavelengths = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/wavelengths.csv')
test_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv',
                           index_col='planet_id')
f_raw_test = f_read_and_preprocess('test', test_adc_info, test_adc_info.index)
a_raw_test = a_read_and_preprocess('test', test_adc_info, test_adc_info.index)
test = feature_engineering(f_raw_test, a_raw_test)

# Load the model
with open(directory + 'model.pickle', 'rb') as f:
    model = pickle.load(f)
with open(directory + 'sigma_pred.pickle', 'rb') as f:
    sigma_pred = pickle.load(f)
    
# Predict
test_pred = model.predict(test)

# Package into submission file
sub_df = postprocessing(test_pred,
                        test_adc_info.index,
                        sigma_pred=np.tile(np.where(test_adc_info[['star']] <= 1, sigma_pred, 0.001), (1, 283)))
display(sub_df)
sub_df.to_csv('submission.csv')
#!head submission.csv

100%|██████████| 1/1 [00:01<00:00,  1.44s/it]


,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,...,sigma_274,sigma_275,sigma_276,sigma_277,sigma_278,sigma_279,sigma_280,sigma_281,sigma_282,sigma_283
planet_id,,,,,,,,,,,,,,,,,,,,,
499191466,0.002819,0.002847,0.002841,0.002836,0.002839,0.002831,0.00283,0.002838,0.002835,0.002831,...,0.000293,0.000293,0.000293,0.000293,0.000293,0.000293,0.000293,0.000293,0.000293,0.000293
